# 01 - BigQuery - Table Data Source
Use BigQuery to load and prepare data for machine learning:


---
## Source Data
`genai-demo-2024.ml_datasets.ulb_fraud_detection`

FraudFlix Technologies is a cutting-edge company focused on making financial transactions safer. Using machine learning, FraudFlix analyzes huge amounts of transaction data to spot and stop fraud as it happens. Born from a hackathon challenge, the company uses a special dataset of European credit card transactions to train its algorithms. What sets FraudFlix apart is its approach to continuously testing and improving its fraud detection models by simulating real-world transactions. This innovative strategy is a game-changer in the fight against digital fraud, offering both businesses and consumers a higher level of security. For data engineers and scientists, FraudFlix represents an exciting frontier where AI meets financial safety, showcasing practical applications of their skills to solve real-world problems.


---
## Setup

inputs:

In [14]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'bootkon-2024'

In [29]:
REGION = 'us-central1'
EXPERIMENT = '01'
SERIES = '01'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'ml_datasets'
BQ_TABLE = 'ulb_fraud_detection'

# Data source for this series of notebooks: Described above
BQ_SOURCE = f'{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}'
BQ_SOURCE


'bootkon-2024.ml_datasets.ulb_fraud_detection'

packages:

In [33]:
from google.cloud import bigquery
from google.cloud import storage
from google.auth import compute_engine, default
import google.auth
import google.auth.transport.requests

clients:

In [17]:
bq = bigquery.Client(project = PROJECT_ID)
gcs = storage.Client(project = PROJECT_ID)

parameters:

In [18]:
BUCKET = PROJECT_ID

### Retrieve and Review a Sample From The Table:
> **Note:** The `LIMIT 5` statement does limit the number of rows returned by BigQuery to 5 but BigQuery still does a full table scan.  If you have a table larger than 1GB and want to limit the rows scanned for a quick review like then then replacing `LIMIT 5` with `TABLESAMPLE SYSTEM (1 PERCENT)` would be more efficient.  For tables under 1GB it will still return the full table.  More on [Table Sampling](https://cloud.google.com/bigquery/docs/table-sampling)

In [19]:
query = f"""
SELECT *
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` TABLESAMPLE SYSTEM (1 PERCENT)
#LIMIT 5
"""
bq.query(query = query).to_dataframe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V22,V23,V24,V25,V26,V27,V28,Amount,Class,Feedback
0,129734.0,-0.198813,0.119392,-2.298735,-3.528636,2.546113,-0.665831,1.333913,-1.335041,1.237487,...,2.033783,-0.170016,-0.234512,-1.053772,-0.789069,0.237179,0.056104,18.0,0,very satisfied.
1,66178.0,-1.185420,0.077870,0.506246,-1.595231,1.936669,4.059180,-0.474315,0.968808,-0.477701,...,-0.673663,-0.193302,0.968871,0.415615,-0.533667,-0.687462,-0.419359,3.5,0,very satisfied.
2,136008.0,-0.136433,0.560053,0.845477,-0.627512,0.634112,0.315149,0.002208,0.205000,0.350898,...,0.140333,-0.068317,-0.031056,-0.462808,0.346058,0.075857,0.091283,12.0,0,very satisfied.
3,127121.0,2.202015,-0.354462,-1.794030,-0.656392,-0.169861,-1.455671,-0.079688,-0.449250,-0.652293,...,0.016810,0.158392,-0.155432,0.010322,-0.274131,-0.017045,-0.035063,15.0,0,very satisfied.
4,132681.0,2.330940,-1.438089,-1.209158,-1.622624,-1.114463,-0.547751,-1.156816,-0.104808,-0.974543,...,-0.203493,0.247454,0.409652,-0.164476,-0.172364,-0.007331,-0.050971,15.0,0,very satisfied.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190919,43222.0,1.258223,0.167619,-0.279844,0.860342,0.801559,1.142473,-0.050596,0.218743,0.081478,...,-0.221026,-0.287056,-1.677702,0.881971,-0.223809,0.038173,-0.011976,1.0,0,Very happy with the quick and efficient service.
190920,45780.0,-0.641757,1.754101,0.365990,1.069202,0.085786,-1.072297,0.515913,0.293245,-0.771182,...,0.280179,-0.145002,0.282845,-0.169995,-0.314329,0.311588,0.193703,1.0,0,Very happy with the quick and efficient service.
190921,49696.0,-7.688513,6.937304,-6.129807,1.398866,-4.860853,-1.687377,-5.103604,6.426214,-1.824883,...,0.125162,0.876694,0.248950,0.271167,-0.394922,-2.738739,-0.638291,1.0,0,Very happy with the quick and efficient service.
190922,51685.0,1.121443,-0.831631,1.633054,0.633309,-1.621314,0.658897,-1.395933,0.546871,2.247837,...,0.074048,0.030859,0.140410,0.116644,1.164445,0.008750,0.008052,1.0,0,Very happy with the quick and efficient service.


### Check out this table in BigQuery Console:
- Click: https://console.cloud.google.com/bigquery
- Make sure project selected is the one from this notebook
- Under Explore, expand this project and review the dataset and table

In [20]:
print(f"Direct Link To This Project In BigQuery:\nhttps://console.cloud.google.com/bigquery?project={PROJECT_ID}")

Direct Link To This Project In BigQuery:
https://console.cloud.google.com/bigquery?project=bootkon-2024


---
## Review Data in BigQuery
Additional SQL queries could be used to review the data.  This section shows moving the table to a Pandas dataframe for local review in Python:

> **Note:** <p>This query only selects one column.  This means BigQuery scans less data as it does not process the other columns.  </p>

In [21]:
query = f"""
SELECT Class
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
"""
df = bq.query(query = query).to_dataframe()

In [22]:
df['Class'].value_counts()

Class
0    566726
1       514
Name: count, dtype: Int64

In [23]:
df['Class'].value_counts(normalize=True)

Class
0    0.999094
1    0.000906
Name: proportion, dtype: Float64

---
## Prepare Data for Analysis

Create a prepped version of the data with test/train splits using SQL DDL:

In [34]:
# Construct a BigQuery client object.
client = bigquery.Client()

# Get the credentials of the current environment, which should be the service account
credentials, project = default()
request = google.auth.transport.requests.Request()
credentials.refresh(request=request)
print(credentials.service_account_email) 
service_account_email = credentials.service_account_email

query = f"""
CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped` AS
WITH add_id AS(SELECT *, GENERATE_UUID() transaction_id FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`)
SELECT *,
    CASE 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 8 THEN "TRAIN" 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 9 THEN "VALIDATE"
        ELSE "TEST"
    END AS splits,
     "{service_account_email}" as service_account_email
FROM add_id
"""
job = bq.query(query = query)
job.result()

112412469323-compute@developer.gserviceaccount.com


In [35]:
(job.ended-job.started).total_seconds()

13.984

In [36]:
if job.estimated_bytes_processed:
    print(f'{job.estimated_bytes_processed/1000000} MB')

158.771085 MB


Review the test/train split:

In [37]:
query = f"""
SELECT splits, count(*) as Count, 100*count(*) / (sum(count(*)) OVER()) as Percentage
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
GROUP BY splits
"""
bq.query(query = query).to_dataframe()

,splits,Count,Percentage
0,VALIDATE,56538,9.967210
1,TEST,56587,9.975848
2,TRAIN,454115,80.056942


Retrieve a subset of the data to a Pandas dataframe:

In [38]:
query = f"""
SELECT * 
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
LIMIT 5
"""
data = bq.query(query = query).to_dataframe()

In [39]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V25,V26,V27,V28,Amount,Class,Feedback,transaction_id,splits,service_account_email
0,127323.0,1.818257,-1.181901,-0.289175,-1.304134,-1.400012,-0.859518,-0.775266,-0.062561,2.476560,...,-0.425863,-0.314204,0.036422,-0.015001,111.879997,0,very satisfied.,eef4e549-a8b7-4441-80d3-92708dc61a43,TRAIN,112412469323-compute@developer.gserviceaccount...
1,127323.0,1.818257,-1.181901,-0.289175,-1.304134,-1.400012,-0.859518,-0.775266,-0.062561,2.476560,...,-0.425863,-0.314204,0.036422,-0.015001,111.879997,0,very satisfied.,a98f9aaf-4e4b-4cbf-ba77-0b6e1fa3bbd7,TRAIN,112412469323-compute@developer.gserviceaccount...
2,67330.0,-1.546480,-0.099199,1.942447,-0.465494,-0.833492,0.047766,-0.313571,0.784546,-1.723019,...,0.281043,-0.384568,-0.058858,-0.056459,98.000000,0,very satisfied.,37e9a5ef-c79c-448e-8bcf-11b007c6e5d7,TRAIN,112412469323-compute@developer.gserviceaccount...
3,67330.0,-1.546480,-0.099199,1.942447,-0.465494,-0.833492,0.047766,-0.313571,0.784546,-1.723019,...,0.281043,-0.384568,-0.058858,-0.056459,98.000000,0,very satisfied.,9bf88594-6a72-45d8-bc7e-20025ba6cd20,TRAIN,112412469323-compute@developer.gserviceaccount...
4,141467.0,-0.695235,0.718616,0.478503,-1.115864,1.230763,0.188936,1.356451,-0.098282,-2.325237,...,1.313624,-0.476009,0.005203,0.034324,39.400002,0,very satisfied.,3e64a3f3-f284-4e37-a68a-51659f6ecc9a,VALIDATE,112412469323-compute@developer.gserviceaccount...
